In [4]:
# |default_exp utils


In [5]:
#| export
import torch
import random
import os
import numpy as np
from fastai.vision.all import Metric
import torch.nn.functional as F

In [6]:
#| export

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
    
def loss(pred,target):
    p = pred[target['mask'][:,:pred.shape[1]]]
    y = target['react'][target['mask']].clip(0,1)
    loss = F.l1_loss(p, y, reduction='none')
    loss = loss[~torch.isnan(loss)].mean()
    
    return loss

def loss_unc(pred,target):
    p = pred[target['mask'][:,:pred.shape[1]]]
    y = target['react'][target['mask']]#.clip(0,1)
    loss = F.l1_loss(p, y, reduction='none')
    loss = loss[~torch.isnan(loss)].mean()
    
    return loss

class MAE(Metric):
    def __init__(self): 
        self.reset()
        
    def reset(self): 
        self.x,self.y = [],[]
        
    def accumulate(self, learn):
        x = learn.pred[learn.y['mask'][:,:learn.pred.shape[1]]].clip(0,1)
        y = learn.y['react'][learn.y['mask']].clip(0,1)
        self.x.append(x)
        self.y.append(y)

    @property
    def value(self):
        x,y = torch.cat(self.x,0),torch.cat(self.y,0)
        loss = F.l1_loss(x, y, reduction='none')
        loss = loss[~torch.isnan(loss)].mean()
        return loss
    
def loss_laplace(pred,target):
    p = pred[target['mask'][:,:pred.shape[1]]].float()
    y = target['react'][target['mask']].clip(0,1).float()
    y_err = target['react_err'][target['mask']].float()
    
    loss = F.l1_loss(p, y, reduction='none')
    m = ~torch.isnan(loss)
    loss = (loss[m]/torch.sqrt(1.0+torch.nan_to_num(y_err[m],100.0).clip(0,100.0))).mean()
    
    return loss

In [7]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()